<a href="https://colab.research.google.com/github/sidagarwal-labs/DSBA-6211---Advance-Business-Analytics/blob/main/Amtrak_Forecasting_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import the dataset and read the data
import pandas as pd
df = pd.read_csv("Amtrak.csv")
df.describe(include='all')

FileNotFoundError: [Errno 2] No such file or directory: 'Amtrak.csv'

In [ ]:
#read the data
df

In [ ]:
df.info

In [ ]:
#convert the data to time series
df['Date'] = pd.to_datetime(df.Month, format='%d/%m/%Y')

In [ ]:
#create rider time series
rider_ts = pd.Series(df.Ridership.values,
                     index=df.Date,
                     name='Ridership')

In [ ]:
rider_ts

In [ ]:
#plot the time series
import matplotlib.pyplot as plt
ax=rider_ts.plot()
ax.set_xlabel('Time')
ax.set_ylabel('Ridership (in 000s)')
ax.set_ylim(1300,2300)
plt.show()

In [ ]:
#we see some seasonality/trend
#intially decreasing, but has started increasing since 1997
#now we will do a time based data partition to focus on recent data points
#train is older data, test is newer data

nValid = 36 #3 years so 12*3 = past 36 months
nTrain=len(rider_ts)-nValid #everything minus the past 36 months is training
train_ts=rider_ts[:nTrain]
valid_ts=rider_ts[nTrain:]

In [ ]:
#lets look at the training dataset
train_ts

In [ ]:
#create regression based model
import statsmodels.formula.api as sm
from statsmodels.tsa import tsatools, stattools

In [ ]:
#create dataframe for regression based models
#add trend for the entire time series (train and test)
#create dummy variable for modeling months
ts_df = tsatools.add_trend(rider_ts, trend='ct')
ts_df['Month']=ts_df.index.month

In [ ]:
ts_df

In [ ]:
#split dataframe
nValid = 36
nTrain=len(ts_df)-nValid
train_df=ts_df[:nTrain]
valid_df=ts_df[nTrain:]

#train first regression model with linear trend
rider_lm=sm.ols(formula='Ridership ~ trend', data=train_df).fit()

In [ ]:
#check the model summary
#notice how trend, Beta 1 has a coeff of 0.35 but the p value is 0.39
#this is greater than 0.05. We expect this since the trend only impacts some of the data.
#the p value for beta 1 will be high and insignificant in linear trends
rider_lm.summary()

In [ ]:
#now lets check model performance on the validation set
! pip install dmba
from dmba import regressionSummary

In [ ]:
#lets check the model performance
predict_lm=rider_lm.predict(valid_df)
regressionSummary(valid_ts,predict_lm)

In [ ]:
#lets create model version 2
#linear trend model with quad, polynomial trend

import numpy as np
rider_lm_poly=sm.ols(formula='Ridership ~ trend + np.square(trend)', data=train_df).fit()

In [ ]:
#now lets look at the model results
#notice now beta 1 and beta 2 are both stat sig
rider_lm_poly.summary()

In [ ]:
#now lets check the model performance
#notice how MAPE is lower (7.08) for this model than linear trend (10.15)
#this is a better model
predict_lm_poly=rider_lm_poly.predict(valid_df)
regressionSummary(valid_ts,predict_lm_poly)

In [ ]:
#create model #3 for seasonality per month
#notice how the P value is under 0.05 for all except month 2, February.
#This means the difference between January vs February is insignifiant. Since 0 is January.
#cannot drop a dummy
rider_lm_season = sm.ols(formula='Ridership~C(Month)', data=train_df).fit()
rider_lm_season.summary()

In [ ]:
#check the model score of seasonality model
#notice how MAPE is higher, so technically without seasonality is better
predict_lm_season=rider_lm_season.predict(valid_df)
regressionSummary(valid_ts,predict_lm_season)

In [ ]:
#create model 4, trend and seasonality
#look at the model summary
modelformula = 'Ridership ~ trend + np.square(trend)+C(Month)'
rider_lm_trendseason = sm.ols(formula=modelformula, data=train_df).fit()
rider_lm_trendseason.summary()

In [ ]:
#now lets check model summary of model 4
#notice how this has the lowest MAPE @ 6.7
predict_lm_trendseason=rider_lm_trendseason.predict(valid_df)
regressionSummary(valid_ts,predict_lm_trendseason)

In [ ]:
#create a simple moving average model based on last 12 months
ma = train_ts.rolling(12).mean()

In [ ]:
#notice how first 11 rows will be blank since there is not enough historic data
ma

In [ ]:
last_ma=ma[-1]

In [ ]:
last_ma

In [ ]:
# Forecasting based on last_ma
predict_ma = pd.Series(last_ma,index=valid_ts.index)
predict_ma

In [ ]:
#summary of stats of simple moving average on actual
regressionSummary(valid_ts, predict_ma)

In [ ]:
#create exponential smoothing model
from statsmodels.tsa.api import SimpleExpSmoothing
SES = SimpleExpSmoothing(train_ts, initialization_method='estimated').fit()

In [ ]:
alphas = [0.1, 0.3, 0.5, 0.7, 0.9]

print("Testing different alpha values for Simple Exponential Smoothing:")
for alpha in alphas:
    print(f"\n--- Alpha = {alpha} ---")
    SES_fixed_alpha = SimpleExpSmoothing(train_ts, initialization_method=None).fit(smoothing_level=alpha)
    predict_SES_fixed_alpha = SES_fixed_alpha.forecast(len(valid_ts))
    print(f"SES model parameters with alpha={alpha}:\n{SES_fixed_alpha.model.params}")
    print("Performance with fixed alpha:")
    regressionSummary(valid_ts, predict_SES_fixed_alpha)
